# Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from re import sub # For price conversions
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,71609,https://www.airbnb.com/rooms/71609,20221229070856,2022-12-29,city scrape,Ensuite Room (Room 1 & 2) near EXPO,For 3 rooms.Book room 1&2 and room 4<br /><br ...,NaN,https://a0.muscache.com/pictures/24453191/3580...,367042,...,4.78,4.26,4.32,NaN,f,6,0,6,0,0.15
1,71896,https://www.airbnb.com/rooms/71896,20221229070856,2022-12-29,city scrape,B&B Room 1 near Airport & EXPO,<b>The space</b><br />Vocational Stay Deluxe B...,NaN,https://a0.muscache.com/pictures/2440674/ac4f4...,367042,...,4.43,4.17,4.04,NaN,t,6,0,6,0,0.17
2,71903,https://www.airbnb.com/rooms/71903,20221229070856,2022-12-29,city scrape,Room 2-near Airport & EXPO,"Like your own home, 24hrs access.<br /><br /><...",Quiet and view of the playground with exercise...,https://a0.muscache.com/pictures/568743/7bc623...,367042,...,4.64,4.50,4.36,NaN,f,6,0,6,0,0.33
3,275343,https://www.airbnb.com/rooms/275343,20221229070856,2022-12-29,city scrape,Amazing Room with window 10min to Redhill,Awesome location and host <br />Room near INSE...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,1439258,...,4.42,4.53,4.63,S0399,f,46,2,44,0,0.19
4,275344,https://www.airbnb.com/rooms/275344,20221229070856,2022-12-29,city scrape,15 mins to Outram MRT Single Room,Lovely home for the special guest !<br /><br /...,Bus stop <br />Food center <br />Supermarket,https://a0.muscache.com/pictures/miso/Hosting-...,1439258,...,4.54,4.62,4.46,S0399,f,46,2,44,0,0.11


URL, id, dates and description can be dropped as they are excess information. \
'host_location' is constant and can be dropped as well.

In [3]:
# Only keep important columns
expr = ['listing_url']
info = ['id', 'name', 'description', 'latitude', 'longitude', 'price', 'property_type', 'room_type', 
        'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities',
        'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'availability_365']
host = ['host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
        'host_total_listings_count', 'host_verification', 'number_of_reviews', 'review_scores_rating',
        'calculated_host_listings_count']
df = df.filter(items= expr + info + host)

In [4]:
# Fraction of Na values
df.isnull().mean()

listing_url                       0.000000
id                                0.000000
name                              0.000000
description                       0.016464
latitude                          0.000000
longitude                         0.000000
price                             0.000000
property_type                     0.000000
room_type                         0.000000
accommodates                      0.000000
bathrooms_text                    0.008890
bedrooms                          0.100428
beds                              0.029635
amenities                         0.000000
neighbourhood_cleansed            0.000000
neighbourhood_group_cleansed      0.000000
availability_365                  0.000000
host_response_rate                0.203820
host_acceptance_rate              0.243662
host_is_superhost                 0.000329
host_listings_count               0.000000
host_total_listings_count         0.000000
number_of_reviews                 0.000000
review_scor

### Group Property Types
There is too many property types to work with.

In [5]:
df[['room_type', 'property_type']].value_counts()

room_type        property_type                     
Private room     Private room in rental unit           589
Entire home/apt  Entire condo                          451
                 Entire rental unit                    377
                 Entire serviced apartment             372
Private room     Private room in condo                 256
                 Room in hotel                         156
                 Private room in home                  135
                 Room in boutique hotel                107
Hotel room       Room in hostel                         64
Private room     Private room in townhouse              52
Shared room      Shared room in hostel                  51
Private room     Private room in serviced apartment     43
Entire home/apt  Room in aparthotel                     39
Private room     Private room in bungalow               31
Hotel room       Room in boutique hotel                 30
Private room     Private room in hostel                 29
    

### Na beds and bedrooms

In [12]:
df[(df['room_type'] == 'Hotel room')
 & df['bedrooms'].isna()][['property_type', 'room_type', 'bedrooms', 'accommodates', 'beds', 'amenities', 'listing_url']]

,property_type,room_type,bedrooms,accommodates,beds,amenities,listing_url
47,Room in bed and breakfast,Hotel room,NaN,6,NaN,"[""Cooking basics"", ""First aid kit"", ""Refrigera...",https://www.airbnb.com/rooms/1678754
48,Room in bed and breakfast,Hotel room,NaN,1,NaN,"[""Cooking basics"", ""Refrigerator"", ""Kitchen"", ...",https://www.airbnb.com/rooms/1678755
75,Room in hostel,Hotel room,NaN,12,NaN,"[""Air conditioning"", ""Fire extinguisher"", ""Sha...",https://www.airbnb.com/rooms/3717196
84,Room in hostel,Hotel room,NaN,12,NaN,"[""Air conditioning"", ""Fire extinguisher"", ""Sha...",https://www.airbnb.com/rooms/4054698
85,Room in hostel,Hotel room,NaN,12,NaN,"[""Air conditioning"", ""Fire extinguisher"", ""Sha...",https://www.airbnb.com/rooms/4069756
...,...,...,...,...,...,...,...
1525,Room in hotel,Hotel room,NaN,2,NaN,"[""Air conditioning"", ""Shampoo"", ""Smoke alarm"",...",https://www.airbnb.com/rooms/38939902
1527,Room in hotel,Hotel room,NaN,2,NaN,"[""Air conditioning"", ""Shampoo"", ""Smoke alarm"",...",https://www.airbnb.com/rooms/38986386
1528,Room in hotel,Hotel room,NaN,2,NaN,"[""Central air conditioning"", ""Shampoo"", ""Smoke...",https://www.airbnb.com/rooms/38986709
1530,Room in hotel,Hotel room,NaN,2,NaN,"[""Air conditioning"", ""Shampoo"", ""Shower gel"", ...",https://www.airbnb.com/rooms/39014250


For hotel room, one listing has 1 bedroom and if 1 accommodate = .

For private and shared room, It is a good assumption that 1 bed = 1 bedroom and 1 accommodate = 1 bed.

In [7]:
# For hotel room
# df['beds'] = df[df['room_type'] == 'Hotel room']['beds'].fillna()
df['bedrooms'] = df[df['room_type'] == 'Hotel room']['bedrooms'].fillna(1) 
# For private room
df['beds'] = df[df['room_type'] == 'Private room']['beds'].fillna(df['accommodates'])
df['bedrooms'] = df[df['room_type'] == 'Private room']['bedrooms'].fillna(df['beds'])
df.isna().mean()

listing_url                       0.000000
id                                0.000000
name                              0.000000
description                       0.016464
latitude                          0.000000
longitude                         0.000000
price                             0.000000
property_type                     0.000000
room_type                         0.000000
accommodates                      0.000000
bathrooms_text                    0.008890
bedrooms                          0.511031
beds                              0.511031
amenities                         0.000000
neighbourhood_cleansed            0.000000
neighbourhood_group_cleansed      0.000000
availability_365                  0.000000
host_response_rate                0.203820
host_acceptance_rate              0.243662
host_is_superhost                 0.000329
host_listings_count               0.000000
host_total_listings_count         0.000000
number_of_reviews                 0.000000
review_scor

### Convert price from string to float

In [8]:
print("Before cleaing:", df.price.count(), df.price.dtype)
df['price'] = df.price.map(lambda x: float(sub(r'[^\d.]', '', x)))
print("After cleaning:", df.price.count(), df.price.dtype)

Before cleaing: 3037 object
After cleaning: 3037 float64


### Filter dataset with a 'trusted' level depending on host reviews

In [9]:
# # Predict whether a listing is legitimate and remove them
# df['estimated_verified'] = (df['number_of_reviews'] > 0)
# display(df['estimated_verified'].value_counts())
# df = df[df['estimated_verified']]
# df.drop(labels=['estimated_verified'], axis=1)

### Trim outliers and $0 rental prices

In [10]:
# Remove $0 rental
df = df.query('price != 0')
# Remove outliers
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df['price'] < (Q1 - 1.5 * IQR)) | (df['price'] > (Q3 + 1.5 * IQR)))]

### Remove host values

In [11]:
df = df.filter(items= expr + info)
df.to_csv('data/clean_listing.csv')